In [1]:
import cv2
import numpy as np
import time
from roboflow import Roboflow

# --- 1. 參數設定區 ---
ROBOFLOW_API_KEY = "UCLBeCClmaD7BW6BWuLG" # <--- 請確認這是您的 API Key
PROJECT_ID = "potato-detection-3et6q"
MODEL_VERSION = 11

# 效能調整：每 N 幀畫面處理一次 (GPU 加速後，可以嘗試調低這個數字，例如 5 或 1)
PROCESS_EVERY_N_FRAMES = 5 

# --- 2. 載入 Roboflow 模型 ---
print("正在載入 Roboflow 模型...")
try:
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    project = rf.workspace().project(PROJECT_ID)
    detection_model = project.version(MODEL_VERSION).model
    print("✅ Roboflow 物件偵測模型載入成功！(將使用 GPU 加速)")
except Exception as e:
    print(f"❌ Roboflow 模型載入失敗: {e}")
    detection_model = None

# --- 3. 主程式邏輯 ---
def find_and_select_camera(max_cameras_to_check=5):
    available_cameras = []
    print("🔍 正在尋找可用的攝影機...")
    for i in range(max_cameras_to_check):
        cap = cv2.VideoCapture(i, cv2.CAP_DSHOW)
        if cap.isOpened():
            print(f"  ✅ 找到攝影機，索引為: {i}")
            available_cameras.append(i)
            cap.release()
        else:
            break
    if not available_cameras: return None
    if len(available_cameras) == 1: return available_cameras[0]
    while True:
        try:
            choice = input(f"\n👉 請從可用的索引 {available_cameras} 中，輸入您想使用的攝影機編號: ")
            selected_index = int(choice)
            if selected_index in available_cameras: return selected_index
            else: print(f"❌ 無效的選擇。")
        except ValueError:
            print("❌ 輸入無效，請輸入一個數字。")

if detection_model:
    selected_cam_index = find_and_select_camera()

    if selected_cam_index is not None:
        cap = cv2.VideoCapture(selected_cam_index, cv2.CAP_DSHOW)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

        if not cap.isOpened():
            print(f"❌ 錯誤：無法打開選擇的攝影機 {selected_cam_index}。")
        else:
            print("\n📹 攝影機已啟動！按下 'q' 鍵退出實時偵測視窗。")
            
            frame_counter = 0
            latest_boxes_and_labels = []

            while True:
                ret, frame = cap.read()
                if not ret: break

                if frame_counter % PROCESS_EVERY_N_FRAMES == 0:
                    latest_boxes_and_labels = []
                    
                    # === 優化點：直接傳送 frame 進行偵測，而不是存成檔案 ===
                    predictions = detection_model.predict(frame, confidence=40, overlap=30).json()['predictions']

                    for pred in predictions:
                        class_name = pred['class']
                        confidence = pred['confidence']
                        
                        label = f"{class_name}: {confidence:.2f}"
                        color = (0, 255, 0) # 預設為綠色 (Fresh)
                        if "sprouted" in class_name.lower():
                            color = (0, 0, 255) # 發芽: 紅色
                        elif "damaged" in class_name.lower():
                            color = (0, 165, 255) # 損傷: 橘色

                        center_x, center_y, width, height = int(pred['x']), int(pred['y']), int(pred['width']), int(pred['height'])
                        x1, y1 = int(center_x - width / 2), int(center_y - height / 2)
                        x2, y2 = int(center_x + width / 2), int(center_y + height / 2)
                        
                        latest_boxes_and_labels.append(((x1, y1, x2, y2), label, color))
                
                # 繪圖邏輯 (使用上一輪的偵測結果，讓畫面感覺流暢)
                for box, label, color in latest_boxes_and_labels:
                    (x1, y1, x2, y2) = box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
                cv2.imshow('Real-time Potato Quality Detection', frame)
                
                frame_counter += 1

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            cap.release()
            cv2.destroyAllWindows()

正在載入 Roboflow 模型...
loading Roboflow workspace...
loading Roboflow project...
✅ Roboflow 物件偵測模型載入成功！(將使用 GPU 加速)
🔍 正在尋找可用的攝影機...
  ✅ 找到攝影機，索引為: 0
  ✅ 找到攝影機，索引為: 1



👉 請從可用的索引 [0, 1] 中，輸入您想使用的攝影機編號:  0



📹 攝影機已啟動！按下 'q' 鍵退出實時偵測視窗。
